In [1]:
# imports

import os
import io
import sys
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [10]:
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
#os.environ['ANTHROPIC_API_KEY'] = os.getenv()
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"



!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [9]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [5]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time. Keep implementations of random number generators identical so that results match exactly."

In [6]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [7]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [8]:
# write to a file called optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [44]:
def optimized_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply 
    write_output(reply)

In [42]:
def optimized_llama(python):    
    payload = {
        "model": MODEL,
        "messages": messages_for(python),
        "stream": True
    }

    stream = requests.post(OLLAMA_API, json=payload, headers=HEADERS, stream=True)
    reply = ""
    for line in stream.iter_lines():
        if line:
            try:
                data = json.loads(line.decode("utf-8"))
                fragment = data.get("message", {}).get("content", "")
                reply += fragment
                yield reply
            except json.JSONDecodeError:
                continue
    write_output(reply)

In [30]:
pi_code = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [16]:
exec(pi_code)

Result: 3.141592658589
Execution Time: 9.243548 seconds


In [34]:
#optimized_gpt(pi_code)
optimized_llama(pi_code)

```cpp
#include <iostream>
#include <chrono>

using namespace std;

// Function to calculate the result of the given formula for a specified number of iterations
double calculate(int iterations, double param1, double param2) {
    // Initialize the result variable with 0.0
    double result = 0.0;
    
    // Perform the calculation for the specified number of iterations
    for (int i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        
        // Subtract the reciprocal of j from the result
        result -= 1 / j;
        
        // Add the reciprocal of j to the result
        j = i * param1 + param2;
        result += 1 / j;
    }
    
    return result;
}

int main() {
    // Start the timer
    auto start_time = chrono::high_resolution_clock::now();
    
    // Calculate the result for 100,000,000 iterations with parameters 4 and 1
    double result = calculate(100_000_000, 4.0, 1.0) * 4.0;
    
    // Stop the timer
    auto end_time = chrono::high_res

In [22]:
!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

Result: 3.141592658589
Execution Time: 0.207829 seconds


In [31]:
def optimize(python, model):
    if model=="GPT":
        result = optimized_gpt(python)
    elif model=="OLLAMA":
        result = optimized_llama(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far 

In [43]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=pi_code)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "OLLAMA"], label="Select model")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=False)

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [46]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [52]:
compiler_cmd = "optimized"
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value="", lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        with gr.Column():
            sample_program = gr.Radio(["pi", "python_hard"], label="Sample program", value="python_hard")
            model = gr.Dropdown(["GPT", "Claude", "CodeQwen"], label="Select model", value="GPT")
        with gr.Column():
            architecture = gr.Radio([compiler_cmd[0]], label="Architecture", interactive=False, value=compiler_cmd[0])
            compiler = gr.Radio([compiler_cmd[1]], label="Compiler", interactive=False, value=compiler_cmd[1])
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        if not compiler_cmd[1] == "Unavailable":
            cpp_run = gr.Button("Run C++")
        else:
            cpp_run = gr.Button("No compiler to run C++", interactive=False)
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    #sample_program.change(select_sample_program, inputs=[sample_program], outputs=[python])
    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(optimize, inputs=[python], outputs=[python_out])
    cpp_run.click(optimize, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

/Users/al/Desktop/LLM Engineering- Master AI, LLM/projects/llm_engineering/venv/lib/python3.12/site-packages/gradio/utils.py:1017: UserWarning: Expected 2 arguments for function <function optimize at 0x16675c7c0>, received 1.
  warnings.warn(
/Users/al/Desktop/LLM Engineering- Master AI, LLM/projects/llm_engineering/venv/lib/python3.12/site-packages/gradio/utils.py:1021: UserWarning: Expected at least 2 arguments for function <function optimize at 0x16675c7c0>, received 1.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


/Users/al/Desktop/LLM Engineering- Master AI, LLM/projects/llm_engineering/venv/lib/python3.12/site-packages/gradio/helpers.py:977: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
Traceback (most recent call last):
  File "/Users/al/Desktop/LLM Engineering- Master AI, LLM/projects/llm_engineering/venv/lib/python3.12/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/al/Desktop/LLM Engineering- Master AI, LLM/projects/llm_engineering/venv/lib/python3.12/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/al/Desktop/LLM Engineering- Master AI, LLM/projects/llm_engineering/venv/lib/python3.12/site-packages/gradio/blocks.py", line 2136, in process_api
    result = await sel